In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from tqdm import tqdm_notebook
import gc

In [2]:
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve

In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [67]:
meta = train.ID_code.to_frame()

In [69]:
cols = train.columns[2:]

In [9]:
for col in cols:
    meta[col + '_square'] = train[col] ** 2

In [72]:
meta= meta.drop(['ID_code'], axis = 1)

In [70]:
for i, col_1 in tqdm_notebook(enumerate(cols)):
    for col_2 in cols:
        meta[col_1 + '_' + col_2] = train[col_1] * train[col_2]
    
    if i == 10:
        break

In [65]:
train = np.around(train, 2)
gc.collect()

643

In [4]:
target = train.target

In [74]:
train = meta

In [75]:
# https://www.kaggle.com/fayzur/customer-transaction-prediction-strong-baseline
# Thanks fayzur. Nice Parameter 
param = {
        'num_leaves': 10,
        'max_bin': 119,
        'min_data_in_leaf': 11,
        'learning_rate': 0.02,
        'min_sum_hessian_in_leaf': 0.00245,
        'bagging_fraction': 1.0, 
        'bagging_freq': 5, 
        'feature_fraction': 0.05,
        'lambda_l1': 4.972,
        'lambda_l2': 2.276,
        'min_gain_to_split': 0.65,
        'max_depth': -1,
        'save_binary': True,
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric': 'auc',
        'is_unbalance': True,
        'boost_from_average': False,
    }

In [76]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))
feature_importance = pd.DataFrame()

train_columns = [c for c in train.columns if c not in ['ID_code', 'target']]
# train_coumns = new_feature

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, target.values)):    
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][train_columns], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][train_columns], label=target.iloc[val_idx])

    num_round = 60000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=3000, early_stopping_rounds = 200)
    oof_lgb[val_idx] = clf.predict(train.iloc[val_idx][train_columns], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(test[train_columns], num_iteration=clf.best_iteration) / folds.n_splits
    
    fold_importance = pd.DataFrame()
    fold_importance["Feature"] = train_columns
    fold_importance["importance"] = clf.feature_importance()
    fold_importance["fold"] = fold_ + 1
    feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
    
    print("CV score: {:<8.5f}".format(roc_auc_score(target.values[val_idx], oof_lgb[val_idx])))
    
print("CV score: {:<8.5f}".format(roc_auc_score(target.values, oof_lgb)))

fold n°0
Training until validation scores don't improve for 200 rounds.
[3000]	training's auc: 0.934408	valid_1's auc: 0.87683
[6000]	training's auc: 0.963717	valid_1's auc: 0.880621
Early stopping, best iteration is:
[6805]	training's auc: 0.969206	valid_1's auc: 0.880972


KeyError: "['var_0_var_0' 'var_0_var_1' 'var_0_var_2' ... 'var_10_var_197'\n 'var_10_var_198' 'var_10_var_199'] not in index"

In [94]:
prob = clf.predict(train.iloc[val_idx][train_columns], num_iteration=clf.best_iteration)

In [102]:
len(prob[prob < 0.5]) / prob.__len__()

0.809604759881003

In [110]:
35981/40000

0.899525

In [109]:
pd.DataFrame(target.values[val_idx])[0].value_counts()

0    35981
1     4020
Name: 0, dtype: int64

In [174]:
result = []
for v in prob:
    if v < 0.431:
        result.append(5)
    else:
        result.append(46)

In [125]:
np.sum(result)

4020

In [121]:
np.quantile(prob, 0.899525)

0.6718763845684592

In [182]:
np.around(prob,2)

array([0.11, 0.19, 0.15, ..., 0.03, 0.24, 0.24])

In [216]:
roc_auc_score(target.values[val_idx], temp.prob)

0.9843695767569975

In [178]:
temp = pd.DataFrame({"target":target.values[val_idx], "prob": prob})

In [196]:
t = temp[temp.target == 1].sort_values('prob').index[:1000]

In [214]:
temp.loc[temp.index.isin(tt), 'prob'] = 0

In [213]:
tt = temp[temp.target == 0].sort_values('prob', ascending=False).index[:1000]

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [63]:
rf = RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42)

In [64]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

oof_rf = np.zeros(len(train))
predictions_rf = np.zeros(len(test))
feature_importance = pd.DataFrame()

train_columns = [c for c in train.columns if c not in ['ID_code', 'target']]
# train_coumns = new_feature

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, target.values)):    
    print("fold n°{}".format(fold_))
    x_train = train.iloc[trn_idx][train_columns]
    y_train = target.iloc[trn_idx]
    x_val = train.iloc[val_idx][train_columns]
    y_val = target.iloc[val_idx]
    
    rf.fit(x_train, y_train)
    
    oof_rf[val_idx] = rf.predict_proba(x_val)[:, 1]
    
    predictions_rf += rf.predict_proba(test[train_columns])[:, 1] / folds.n_splits
    
    fold_importance = pd.DataFrame()
    fold_importance["Feature"] = train_columns
    fold_importance["importance"] = rf.feature_importances_
    fold_importance["fold"] = fold_ + 1
    feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
    
    print("CV score: {:<8.5f}".format(roc_auc_score(target.values[val_idx], oof_rf[val_idx])))
    
print("CV score: {:<8.5f}".format(roc_auc_score(target.values, oof_rf)))

fold n°0
CV score: 0.83793 
fold n°1


KeyboardInterrupt: 

In [51]:
roc_auc_score(y_val.values, rf.predict_proba(x_val)[:,1])

0.7468904720365922

In [53]:
rf.predict_proba(x_val)[:,1]

array([0.3 , 0.1 , 0.05, ..., 0.15, 0.3 , 0.15])

In [42]:
y_val.value_counts()

0    35980
1     4019
Name: target, dtype: int64

In [39]:
rf.predict(x_val)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [40]:
pd.DataFrame(rf.predict(x_val))[0].value_counts()

0    39894
1      105
Name: 0, dtype: int64

In [17]:
train.target.value_counts()

0    179902
1     20098
Name: target, dtype: int64

In [21]:
pd.DataFrame((predictions_lgb >= 0.65) * 1)[0].value_counts()

0    177481
1     22519
Name: 0, dtype: int64

In [18]:
feature_importance.max()

Feature       var_9_var_99
importance             252
fold                     5
dtype: object

In [20]:
new_feature = feature_importance.groupby('Feature').importance.mean().sort_values(ascending = False)[:1000].index

In [19]:
feature_importance.groupby('Feature').importance.mean().sort_values(ascending = False)[:200]

Feature
var_108          247.6
var_12           237.2
var_34           228.6
var_133          219.8
var_166          208.2
var_81           199.8
var_169          199.2
var_53           197.8
var_121          190.4
var_91           183.2
var_148          183.0
var_177          173.4
var_94           164.4
var_197          159.0
var_130          155.0
var_165          152.0
var_198          143.4
var_125          140.4
var_33           138.2
var_109          137.8
var_43           134.2
var_93           133.8
var_56           128.0
var_92           122.0
var_150          119.6
var_78           119.0
var_18           115.0
var_146          115.0
var_184          113.4
var_188          108.6
                 ...  
var_4_var_165     42.4
var_6_var_80      42.2
var_37            42.0
var_4_var_110     42.0
var_2_var_13      41.8
var_76            41.8
var_192           41.6
var_3_var_13      41.6
var_59            41.6
var_0_var_139     41.6
var_0_var_147     41.6
var_7_var_32      41.4
var